In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import glob
import pickle

gp_drt_dir = '../comparisons/GP-DRT'
if gp_drt_dir not in sys.path:
    sys.path.append(gp_drt_dir)
import GP_DRT as gp
from GP_utils import gp_fit
    
if '../../../../misc_modules' not in sys.path:
    sys.path.append('../../../../misc_modules')
import gamry_tools as gt

%load_ext autoreload
%autoreload 2

reloaded GP_DRT
reloaded GP_utils


In [2]:
def save_pickle(obj, file):
    with open(file,'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
    print('Dumped pickle to {}'.format(file))
    
def load_pickle(file):
    with open(file,'rb') as f:
        return pickle.load(f)

# Simulated data

In [13]:
# tau for plotting
tau_plot = np.logspace(-7,2,200)
f_plot = 1/(2*np.pi*tau_plot)
start = time.time()
files = glob.glob('../../data/simulated/Z*.csv')
error_files = []
theta0s = [[0.1,0.5,1],[0.1,1.5,2],[0.1,5,1]]
for file in files:
    print(file)
    print('-------------------------')
    suffix = file[file.find('_'):-4]
    file_start = time.time()
    best_path = 'results/for_figs'
    Zoutbest = os.path.join(best_path,f'Zout{suffix}.csv')
    Goutbest = os.path.join(best_path,f'Gout{suffix}.csv')
    pklbest = os.path.join(best_path,f'obj_{suffix}.pkl')
    if os.path.exists(Zoutbest) and os.path.exists(Goutbest) and os.path.exists(pklbest):
        print('Already ran')
    else:
    
        best_fun = np.inf
        for n,theta0 in enumerate(theta0s):
            print('theta0=',theta0)
            respath = 'results/theta0=({},{},{})'.format(theta0[0],theta0[1],theta0[2])
            Zoutfile = os.path.join(respath,f'Zout{suffix}.csv')
            Goutfile = os.path.join(respath,f'Gout{suffix}.csv')
            pkl = os.path.join(respath,f'obj_{suffix}.pkl')
    #         if os.path.exists(Zoutfile) and os.path.exists(Goutfile):
    #             print('Already ran')
    #         else:
            df = pd.read_csv(file)
            # sort ascending
            df = df.sort_values('Freq')
            Z = df['Zreal'].values + 1j*df['Zimag'].values

            try:
                result = gp_fit(df['Freq'].values,Z,theta0=np.array(theta0),freq_star=f_plot,max_iter=20)

                Z_res = pd.DataFrame(np.array([df['Freq'],result['Z_re_fit'],result['Z_im_fit'],result['sigma_Z_im_fit']]).T,
                                     columns=['freq','Zreal','Zimag','sigma_im'])
                g_res = pd.DataFrame(np.array([tau_plot,result['gamma_star'],result['sigma_gamma_star']]).T,
                                     columns=['tau','gamma','sigma_gamma'])
                Z_res.to_csv(Zoutfile,index=False)
                g_res.to_csv(Goutfile,index=False)
                save_pickle(result,pkl)
                # if this theta0 improved the NMLL, write it to for_figs (overwrite existing file)
                if result['min_result']['fun'] < best_fun:
                    best_fun = result['min_result']['fun']
                    Z_res.to_csv(Zoutbest,index=False)
                    g_res.to_csv(Goutbest,index=False)
                    save_pickle(result,pklbest)
            except np.linalg.LinAlgError:
                error_files.append(file)
                print('LinAlgError')
    
        print('File fit time:', time.time()-file_start)
        
print('Total fit time:',time.time()-start)


reloaded GP_utils
../../data/simulated\Z_2RC_Macdonald_0.25.csv
-------------------------
Already ran
../../data/simulated\Z_2RC_Macdonald_1.0.csv
-------------------------
Already ran
../../data/simulated\Z_2RC_Macdonald_2.5.csv
-------------------------
Already ran
../../data/simulated\Z_2RC_noiseless.csv
-------------------------
Already ran
../../data/simulated\Z_2RC_Orazem_0.25.csv
-------------------------
Already ran
../../data/simulated\Z_2RC_Orazem_1.0.csv
-------------------------
Already ran
../../data/simulated\Z_2RC_Orazem_2.5.csv
-------------------------
Already ran
../../data/simulated\Z_2RC_uniform_0.25.csv
-------------------------
Already ran
../../data/simulated\Z_2RC_uniform_1.0.csv
-------------------------
Already ran
../../data/simulated\Z_2RC_uniform_2.5.csv
-------------------------
Already ran
../../data/simulated\Z_2ZARC_Macdonald_0.25.csv
-------------------------
Already ran
../../data/simulated\Z_2ZARC_Macdonald_1.0.csv
-------------------------
Already r

-0.0242197	4.9960599	 1.0171696  -182.108
-0.0244103	4.9960168	 1.0173710  -182.112
-0.0242147	4.9959775	 1.0175545  -182.116
-0.0244046	4.9959367	 1.0177448  -182.120
-0.0242101	4.9958954	 1.0179375  -182.123
-0.0243998	4.9958566	 1.0181190  -182.127
-0.0242059	4.9958134	 1.0183206  -182.131
-0.0243958	4.9957761	 1.0184951  -182.135
         Current function value: -182.134850
         Iterations: 20
         Function evaluations: 26
         Gradient evaluations: 117
         Hessian evaluations: 0
Dumped pickle to results/theta0=(0.1,5,1)\obj__2ZARC_Orazem_1.0.pkl
File fit time: 574.8411395549774
../../data/simulated\Z_2ZARC_Orazem_2.5.csv
-------------------------
theta0= [0.1, 0.5, 1]
sigma_n,	  sigma_f,	 ell,		NMLL
0.0693835	0.4984963	 1.0001036  -160.755
0.0616238	0.4978237	 1.0001702  -162.267
0.0584308	0.4970005	 1.0002576  -162.411
0.0596412	0.4955880	 1.0004112  -162.442
0.0583642	0.4944065	 1.0005380  -162.474
0.0595576	0.4931641	 1.0006722  -162.504
0.0583120	0.4918609	 1.

Dumped pickle to results/theta0=(0.1,1.5,2)\obj__2ZARC_uniform_1.0.pkl
Dumped pickle to results/for_figs\obj__2ZARC_uniform_1.0.pkl
theta0= [0.1, 5, 1]
sigma_n,	  sigma_f,	 ell,		NMLL
-0.0198958	4.9992767	 1.0022886  -192.556
-0.0203215	4.9992633	 1.0023537  -192.586
-0.0202020	4.9490144	 1.2453574  -197.465
-0.0203402	4.8319395	 1.7851921  -205.245
-0.0208300	4.8319299	 1.7852286  -205.286
-0.0209253	4.8318923	 1.7853737  -205.287
-0.0208288	4.8318767	 1.7854341  -205.288
-0.0209221	4.8318419	 1.7855686  -205.290
-0.0208276	4.8318258	 1.7856309  -205.291
-0.0209191	4.8317933	 1.7857564  -205.292
-0.0208264	4.8317766	 1.7858208  -205.293
-0.0209165	4.8317461	 1.7859385  -205.294
-0.0208252	4.8317290	 1.7860051  -205.296
-0.0209142	4.8317002	 1.7861161  -205.297
-0.0208240	4.8316824	 1.7861850  -205.298
-0.0209120	4.8316552	 1.7862901  -205.299
-0.0208228	4.8316368	 1.7863614  -205.300
-0.0209100	4.8316110	 1.7864612  -205.301
-0.0208215	4.8315919	 1.7865351  -205.303
-0.0209082	4.83156

Dumped pickle to results/theta0=(0.1,5,1)\obj__Gerischer_Macdonald_1.0.pkl
File fit time: 362.11094856262207
../../data/simulated\Z_Gerischer_Macdonald_2.5.csv
-------------------------
theta0= [0.1, 0.5, 1]
sigma_n,	  sigma_f,	 ell,		NMLL
-0.0064960	0.4962868	 1.0001863  -306.380
-0.0065352	0.4962744	 1.0001895  -306.383
-0.0065002	0.4961527	 1.0002207  -306.385
-0.0065359	0.4961399	 1.0002240  -306.387
-0.0065026	0.4960426	 1.0002490  -306.390
-0.0065363	0.4960295	 1.0002523  -306.391
-0.0065042	0.4959447	 1.0002741  -306.393
-0.0065366	0.4959313	 1.0002775  -306.395
-0.0065055	0.4958546	 1.0002972  -306.397
-0.0065369	0.4958411	 1.0003007  -306.398
-0.0065065	0.4957701	 1.0003188  -306.400
-0.0065371	0.4957564	 1.0003224  -306.401
-0.0065073	0.4956897	 1.0003395  -306.403
-0.0065373	0.4956758	 1.0003430  -306.404
-0.0065079	0.4956125	 1.0003592  -306.405
-0.0065375	0.4955984	 1.0003628  -306.407
-0.0065085	0.4955378	 1.0003784  -306.408
-0.0065376	0.4955236	 1.0003820  -306.409
-0.0

C:\Users\jdh10\OneDrive - Colorado School of Mines\python\projects\bayes_drt\comparisons\GP-DRT\GP_utils.py:84: RuntimeWarning: invalid value encountered in sqrt
  # estimate Z_im for xi_vec (added by J. Huang)
C:\Users\jdh10\OneDrive - Colorado School of Mines\python\projects\bayes_drt\comparisons\GP-DRT\GP_utils.py:140: RuntimeWarning: invalid value encountered in sqrt
  # estimate Z_re for freq using fine DRT (added by J. Huang)


...

sigma_n,	  sigma_f,	 ell,		NMLL
-0.0028555	4.9994015	 1.0019223  -306.979
-0.0028379	4.9994012	 1.0019253  -306.981
Optimization terminated successfully.
         Current function value: -306.981269
         Iterations: 2
         Function evaluations: 11
         Gradient evaluations: 16
         Hessian evaluations: 0
Dumped pickle to results/theta0=(0.1,5,1)\obj__ZARC-RL_Orazem_0.25.pkl
File fit time: 278.0580463409424
../../data/simulated\Z_ZARC-RL_Orazem_1.0.csv
-------------------------
theta0= [0.1, 0.5, 1]
sigma_n,	  sigma_f,	 ell,		NMLL
-0.0122245	0.4960691	 1.0002065  -268.508
-0.0117835	0.4960281	 1.0002121  -268.576
-0.0118520	0.4959672	 1.0002194  -268.578
-0.0117825	0.4958877	 1.0002293  -268.580
-0.0118513	0.4958282	 1.0002365  -268.582
-0.0117817	0.4957463	 1.0002466  -268.585
-0.0118506	0.4956881	 1.0002536  -268.587
-0.0117808	0.4956039	 1.0002640  -268.589
-0.0118501	0.4955467	 1.0002709  -268.591
-0.0117801	0.4954603	 1.0002816  -268.593
-0.0118496	0.4954041	 1.0002

Dumped pickle to results/theta0=(0.1,0.5,1)\obj__ZARC-RL_uniform_0.25.pkl
Dumped pickle to results/for_figs\obj__ZARC-RL_uniform_0.25.pkl
theta0= [0.1, 1.5, 2]
sigma_n,	  sigma_f,	 ell,		NMLL
-0.0027047	1.4988712	 2.0008845  -358.902
-0.0027074	1.4988706	 2.0008847  -358.902
Optimization terminated successfully.
         Current function value: -358.901788
         Iterations: 2
         Function evaluations: 7
         Gradient evaluations: 12
         Hessian evaluations: 0
Dumped pickle to results/theta0=(0.1,1.5,2)\obj__ZARC-RL_uniform_0.25.pkl
theta0= [0.1, 5, 1]
sigma_n,	  sigma_f,	 ell,		NMLL
-0.0023647	4.9994045	 1.0019136  -317.327
-0.0023693	4.9994037	 1.0019206  -317.327
Optimization terminated successfully.
         Current function value: -317.327230
         Iterations: 2
         Function evaluations: 12
         Gradient evaluations: 19
         Hessian evaluations: 0
Dumped pickle to results/theta0=(0.1,5,1)\obj__ZARC-RL_uniform_0.25.pkl
File fit time: 267.204486846923

sigma_n,	  sigma_f,	 ell,		NMLL
0.0005912	0.4965493	 1.0001855  -450.966
0.0005915	0.4965489	 1.0001857  -450.966
Optimization terminated successfully.
         Current function value: -450.966181
         Iterations: 2
         Function evaluations: 13
         Gradient evaluations: 20
         Hessian evaluations: 0
Dumped pickle to results/theta0=(0.1,0.5,1)\obj__ZARC_Macdonald_0.25.pkl
Dumped pickle to results/for_figs\obj__ZARC_Macdonald_0.25.pkl
theta0= [0.1, 1.5, 2]
sigma_n,	  sigma_f,	 ell,		NMLL
-0.0008309	1.4988914	 2.0008734  -425.644
-0.0004952	1.5101718	 1.5963042  -437.359
-0.0006305	1.5101717	 1.5963042  -441.016
-0.0006247	1.5101717	 1.5963043  -441.022
Optimization terminated successfully.
         Current function value: -441.021799
         Iterations: 4
         Function evaluations: 14
         Gradient evaluations: 29
         Hessian evaluations: 0
Dumped pickle to results/theta0=(0.1,1.5,2)\obj__ZARC_Macdonald_0.25.pkl
theta0= [0.1, 5, 1]
sigma_n,	  sigma_f,	 el

sigma_n,	  sigma_f,	 ell,		NMLL
-0.0023477	4.9994046	 1.0019140  -316.720
-0.0027172	4.8494062	 1.9041465  -341.668
-0.0025071	4.8494059	 1.9041479  -341.944
-0.0025337	4.8494058	 1.9041487  -341.951
Optimization terminated successfully.
         Current function value: -341.951265
         Iterations: 4
         Function evaluations: 14
         Gradient evaluations: 29
         Hessian evaluations: 0
Dumped pickle to results/theta0=(0.1,5,1)\obj__ZARC_Orazem_0.25.pkl
File fit time: 314.02161931991577
../../data/simulated\Z_ZARC_Orazem_1.0.csv
-------------------------
theta0= [0.1, 0.5, 1]
sigma_n,	  sigma_f,	 ell,		NMLL
-0.0098950	0.4961443	 1.0002111  -281.421
-0.0097849	0.4961147	 1.0002199  -281.428
-0.0098654	0.4959153	 1.0002793  -281.432
-0.0097830	0.4958839	 1.0002887  -281.437
-0.0098547	0.4957333	 1.0003335  -281.440
-0.0097814	0.4957005	 1.0003433  -281.444
-0.0098479	0.4955765	 1.0003802  -281.447
-0.0097799	0.4955423	 1.0003904  -281.449
-0.0098430	0.4954355	 1.0004222  

Dumped pickle to results/theta0=(0.1,0.5,1)\obj__ZARC_uniform_0.25.pkl
Dumped pickle to results/for_figs\obj__ZARC_uniform_0.25.pkl
theta0= [0.1, 1.5, 2]
sigma_n,	  sigma_f,	 ell,		NMLL
-0.0024450	1.4988732	 2.0008867  -364.558
-0.0024225	1.4988729	 2.0008862  -364.563
Optimization terminated successfully.
         Current function value: -364.562676
         Iterations: 2
         Function evaluations: 8
         Gradient evaluations: 13
         Hessian evaluations: 0
Dumped pickle to results/theta0=(0.1,1.5,2)\obj__ZARC_uniform_0.25.pkl
theta0= [0.1, 5, 1]
sigma_n,	  sigma_f,	 ell,		NMLL
-0.0022378	4.9994053	 1.0019116  -320.026
-0.0022812	4.9479829	 1.3865452  -334.393
-0.0022509	4.9479827	 1.3865468  -334.403
-0.0022483	4.9215605	 1.5870975  -340.624
-0.0022521	4.9215604	 1.5870988  -340.624
Optimization terminated successfully.
         Current function value: -340.624224
         Iterations: 5
         Function evaluations: 20
         Gradient evaluations: 42
         Hessian e

# Experimental data
## LIB

In [19]:
lib_files = ['../../data/experimental/DRTtools_LIB_data.txt',
             '../../data/experimental/DRTtools_LIB_data_qtr.csv'
            ]

theta0s = [[1e-4,1e-3,1],[5e-4,1e-3,1],[1e-4,2.5e-3,1],[1e-4,1e-3,2],[1e-4,5e-4,2],[1e-4,5e-4,1],
           [1e-4,5e-4,0.5],[1e-4,1e-3,0.5]]

f_plot = np.logspace(4,-5,200)
tau_plot = 1/(2*np.pi*f_plot)

start = time.time()
error_files = []
for file in lib_files:
    print(file)
    print('-------------------------')
    suffix = file[file.find('_'):-4]
    best_path = 'results/for_figs'
    Zoutbest = os.path.join(best_path,f'Zout{suffix}.csv')
    Goutbest = os.path.join(best_path,f'Gout{suffix}.csv')
    pklbest = os.path.join(best_path,f'obj{suffix}.pkl')
    
    if os.path.exists(Zoutbest) and os.path.exists(Goutbest) and os.path.exists(pklbest):
            print('Already ran')
    else:
        best_fun = np.inf
        file_start = time.time()
        for n,theta0 in enumerate(theta0s):
            print('theta0=',theta0)
            tsuf = suffix + '_theta0=({},{},{})'.format(theta0[0],theta0[1],theta0[2])
            Zoutfile = os.path.join('results/exp',f'Zout{tsuf}.csv')
            Goutfile = os.path.join('results/exp',f'Gout{tsuf}.csv')
            pkl = os.path.join('results/exp',f'obj{tsuf}.pkl')
            
            if file[-3:]=='txt':
                df = pd.read_csv(file,sep='\t',header=None)
                df = pd.DataFrame(df.values,columns=['Freq','Zreal','Zimag'])
            else:
                df = pd.read_csv(file)
            # sort ascending
            df = df.sort_values('Freq')
            Z = df['Zreal'].values + 1j*df['Zimag'].values
            try:
                result = gp_fit(df['Freq'].values,Z,theta0=theta0,freq_star=f_plot)
                Z_res = pd.DataFrame(np.array([df['Freq'],result['Z_re_fit'],result['Z_im_fit'],result['sigma_Z_im_fit']]).T,
                                 columns=['freq','Zreal','Zimag','sigma_im'])
                # sort by descending freq
                Z_res = Z_res.sort_values('freq',ascending=False,ignore_index=True)
                g_res = pd.DataFrame(np.array([tau_plot,result['gamma_star'],result['sigma_gamma_star']]).T,
                                     columns=['tau','gamma','sigma_gamma'])

                # save results files
                Z_res.to_csv(Zoutfile,index=False)
                g_res.to_csv(Goutfile,index=False)
                save_pickle(result,pkl)

                # if this theta0 improved the NMLL, write it to for_figs (overwrite existing file)
                if result['min_result']['fun'] < best_fun:
                    best_fun = result['min_result']['fun']
                    Z_res.to_csv(Zoutbest,index=False)
                    g_res.to_csv(Goutbest,index=False)
                    save_pickle(result,pklbest)
            except np.linalg.LinAlgError as lae:
                # cholesky failed for theta0. Can't evaluate NMLL, so no use in fitting
                print(lae)
                print('Can\'t evaluate NMLL for theta0. Skipping')
            
        print('File fit time:', time.time()-file_start)
        
print('Total fit time:',time.time()-start)

../../data/experimental/DRTtools_LIB_data.txt
-------------------------
theta0= [0.0001, 0.001, 1]
sigma_n,	  sigma_f,	 ell,		NMLL
-0.0004366	0.0028997	 0.9999998  -778.814
Matrix is not positive definite
Choosing iteration with lowest NMLL: 1
Dumped pickle to results/exp\obj_LIB_data_theta0=(0.0001,0.001,1).pkl
Dumped pickle to results/for_figs\obj_LIB_data.pkl
theta0= [0.0005, 0.001, 1]
sigma_n,	  sigma_f,	 ell,		NMLL
-0.0009916	0.0042089	 1.0000007  -710.461
Matrix is not positive definite
Choosing iteration with lowest NMLL: 1
Dumped pickle to results/exp\obj_LIB_data_theta0=(0.0005,0.001,1).pkl
theta0= [0.0001, 0.0025, 1]
Matrix is not positive definite
Cant evaluate NMLL for theta0. Skipping
theta0= [0.0001, 0.001, 2]
sigma_n,	  sigma_f,	 ell,		NMLL
0.0070414	0.0016639	 1.9999994  -516.914
Matrix is not positive definite
Choosing iteration with lowest NMLL: 1
Dumped pickle to results/exp\obj_LIB_data_theta0=(0.0001,0.001,2).pkl
theta0= [0.0001, 0.0005, 2]
sigma_n,	  sigma_f,	 ell

## Protonic ceramic microelectrode

In [22]:
tco_file = '../../data/experimental/PDAC_COM3_02109_Contact10_2065C_500C.txt'
tco_df = gt.read_eis_zdata(tco_file)

f_plot_tco = np.logspace(7,-3,200)
tau_plot_tco = 1/(2*np.pi*f_plot_tco)

theta0s = [[5e5,1e6,1],[1e6,1e6,1],[1e6,2.5e6,1],[5e5,2.5e6,1],
          [5e5,1e6,2],[5e5,2.5e6,2],[1e6,1e6,2],[5e5,2.5e6,0.5]]

file = tco_file
suffix = '_PDAC'
Zoutbest = os.path.join('results/for_figs',f'Zout{suffix}.csv')
Goutbest = os.path.join('results/for_figs',f'Gout{suffix}.csv')
pklbest = os.path.join('results/for_figs',f'obj{suffix}.csv')

best_fun = np.inf
file_start = time.time()
for n,theta0 in enumerate(theta0s):
    print('theta0=',theta0)
    tsuf = suffix + '_theta0=({},{},{})'.format(theta0[0],theta0[1],theta0[2])
    Zoutfile = os.path.join('results/exp',f'Zout{tsuf}.csv')
    Goutfile = os.path.join('results/exp',f'Gout{tsuf}.csv')
    pkl = os.path.join('results/exp',f'obj{tsuf}.pkl')
    
    df = gt.read_eis_zdata(tco_file)
    # sort ascending
    df = df.sort_values('Freq')
    Z = df['Zreal'].values + 1j*df['Zimag'].values
    
    try:
        result = gp_fit(df['Freq'].values,Z,theta0=theta0,max_iter=10,freq_star=f_plot_tco)

        Z_res = pd.DataFrame(np.array([df['Freq'],result['Z_re_fit'],result['Z_im_fit'],result['sigma_Z_im_fit']]).T,
                             columns=['freq','Zreal','Zimag','sigma_im'])
        g_res = pd.DataFrame(np.array([tau_plot_tco,result['gamma_star'],result['sigma_gamma_star']]).T,
                             columns=['tau','gamma','sigma_gamma'])
        # sort by descending freq
        Z_res = Z_res.sort_values('freq',ascending=False,ignore_index=True)

        #save files
        Z_res.to_csv(Zoutfile,index=False)
        g_res.to_csv(Goutfile,index=False)
        save_pickle(result,pkl)
        
        # if this theta0 improved the NMLL, write it to for_figs (overwrite existing file)
        if result['min_result']['fun'] < best_fun:
            best_fun = result['min_result']['fun']
            Z_res.to_csv(Zoutbest,index=False)
            g_res.to_csv(Goutbest,index=False)
            save_pickle(result,pklbest)
    except np.linalg.LinAlgError as lae:
        # cholesky failed for theta0. Can't evaluate NMLL, so no use in fitting
        print(lae)
        print('Can\'t evaluate NMLL for theta0. Skipping')
    
        
print('File fit time:', time.time()-file_start)

theta0= [500000.0, 1000000.0, 1]
sigma_n,	  sigma_f,	 ell,		NMLL
499999.9999773	1000000.0000009	 1.4992727  1415.252
499999.9999121	1000000.0000029	 2.0608005  1414.675
499999.9998102	1000000.0000060	 2.3661658  1414.610
499999.9997546	1000000.0000079	 2.3032102  1414.604
499999.9996921	1000000.0000099	 2.2993189  1414.604
Matrix is not positive definite
Choosing iteration with lowest NMLL: 5
Dumped pickle to results/exp\obj_PDAC_theta0=(500000.0,1000000.0,1).pkl
Dumped pickle to results/for_figs\obj_PDAC.csv
theta0= [1000000.0, 1000000.0, 1]
sigma_n,	  sigma_f,	 ell,		NMLL
999999.9999807	1000000.0000009	 1.5752927  1480.652
999999.9999249	1000000.0000027	 2.3310677  1480.186
999999.9998802	1000000.0000043	 2.2677631  1480.181
999999.9998341	1000000.0000060	 2.2674560  1480.181
999999.9978204	1000000.0000786	 2.2677604  1480.181
999999.9977742	1000000.0000803	 2.2674560  1480.181
999997.8765889	1000000.0765874	 2.2674560  1480.181
999997.8745559	1000000.0766607	 2.2677650  1480.181
999